In [1]:
'''
在大改模型后训练模型的时段同步进行可视化部分的修改
主要目标是加入CRF
'''

'\n在大改模型后训练模型的时段同步进行可视化部分的修改\n主要目标是加入CRF\n'

In [2]:
'''
统一设置地址
'''

import os

# 获取当前工作目录
current_dir = os.getcwd()
print("当前工作目录：", current_dir)

# 修改当前工作目录，以后输出文件只需要写文件名
new_dir = "D:/李娅宁/肩台外侧点-0715/"
os.chdir(new_dir)
print("修改后的工作目录：", os.getcwd())


当前工作目录： C:\Users\HP
修改后的工作目录： D:\李娅宁\肩台外侧点-0715


In [3]:
import pkg_resources

installed_packages = pkg_resources.working_set
for package in installed_packages:
    print(f"{package.project_name}=={package.version}")


Automat==20.2.0
Babel==2.11.0
Bottleneck==1.3.5
ConfigArgParse==1.7
Cython==3.0.10
Flask==2.2.2
HeapDict==1.0.1
Jinja2==3.1.2
Markdown==3.4.1
MarkupSafe==2.1.1
Pillow==10.0.1
Protego==0.1.16
PyDispatcher==2.0.5
PyJWT==2.4.0
PyNaCl==1.5.0
PyQt5==5.15.7
PyQt5-sip==12.11.0
PyQtWebEngine==5.15.4
PySocks==1.7.1
PyWavelets==1.4.1
PyYAML==6.0
Pygments==2.15.1
QDarkStyle==3.0.2
QtAwesome==1.2.2
QtPy==2.2.0
Rtree==1.0.1
SQLAlchemy==1.4.39
Scrapy==2.8.0
Send2Trash==1.8.0
Sphinx==5.0.2
TBB==0.2
Twisted==22.10.0
Unidecode==1.2.0
Werkzeug==2.2.3
absl-py==2.1.0
aiobotocore==2.5.0
aiofiles==22.1.0
aiohttp==3.8.5
aioitertools==0.7.1
aiosignal==1.2.0
aiosqlite==0.18.0
alabaster==0.7.12
anaconda-anon-usage==0.4.2
anaconda-catalogs==0.2.0
anaconda-client==1.12.1
anaconda-cloud-auth==0.1.3
anaconda-navigator==2.5.2
anaconda-project==0.11.1
anyio==3.5.0
appdirs==1.4.4
argon2-cffi==21.3.0
argon2-cffi-bindings==21.2.0
arrow==1.2.3
astroid==2.14.2
astropy==5.1
asttokens==2.0.5
astunparse==1.6.3
async-timeout=

In [4]:
import tensorflow as tf
import keras

print("TensorFlow 版本:", tf.__version__)
print("Keras 版本:", keras.__version__)


TensorFlow 版本: 2.17.0
Keras 版本: 3.4.1


In [5]:
# 定义UNET模型
def unet_3d(input_shape):
    inputs = tf.keras.layers.Input(shape=input_shape)

    # Encoder
    conv1 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(inputs)
    conv1 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv1)
    pool1 = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(conv1)

    conv2 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(pool1)
    conv2 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv2)
    pool2 = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(conv2)

    conv3 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(pool2)
    conv3 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv3)
    pool3 = tf.keras.layers.MaxPooling3D(pool_size=(2, 2, 2))(conv3)

    # Bottleneck
    conv4 = tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(pool3)
    conv4 = tf.keras.layers.Conv3D(512, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv4)

    # Decoder
    up5 = tf.keras.layers.Conv3DTranspose(256, 2, strides=(2, 2, 2), padding='same')(conv4)
    concat5 = tf.keras.layers.concatenate([up5, conv3], axis=-1)
    conv5 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(concat5)
    conv5 = tf.keras.layers.Conv3D(256, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv5)

    up6 = tf.keras.layers.Conv3DTranspose(128, 2, strides=(2, 2, 2), padding='same')(conv5)
    concat6 = tf.keras.layers.concatenate([up6, conv2], axis=-1)
    conv6 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(concat6)
    conv6 = tf.keras.layers.Conv3D(128, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv6)

    up7 = tf.keras.layers.Conv3DTranspose(64, 2, strides=(2, 2, 2), padding='same')(conv6)
    concat7 = tf.keras.layers.concatenate([up7, conv1], axis=-1)
    conv7 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(concat7)
    conv7 = tf.keras.layers.Conv3D(64, 3, activation='relu', padding='same', kernel_regularizer=tf.keras.regularizers.l2(1e-4))(conv7)

    # 修改输出层，适应3个类别
    outputs = tf.keras.layers.Conv3D(3, 1, activation='softmax')(conv7)  # 3 channels for three-class classification

    model = tf.keras.models.Model(inputs=[inputs], outputs=[outputs])
    return model

'''
定义各种损失函数
'''

import tensorflow.keras.backend as K
import numpy as np
import tensorflow as tf
from scipy.spatial.distance import cdist


# Focal
def focal_loss(gamma=2.0, alpha=0.25):
    def focal_loss_fixed(y_true, y_pred):
        # 确保 y_true 和 y_pred 都是 float32 类型
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.clip_by_value(y_pred, K.epsilon(), 1 - K.epsilon())
        # 计算交叉熵
        cross_entropy = -y_true * tf.math.log(y_pred)
        # 计算 focal 权重
        focal_weight = alpha * tf.math.pow(1 - y_pred, gamma)
        # 计算 focal loss
        loss = focal_weight * cross_entropy
        return tf.reduce_mean(tf.reduce_sum(loss, axis=-1))
    return focal_loss_fixed


# Tversky
def tversky_loss(y_true, y_pred, alpha=0.7, beta=0.3, smooth=1e-6):
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    intersection = tf.reduce_sum(y_true * y_pred, axis=[1, 2, 3, 4])
    false_neg = tf.reduce_sum(y_true * (1 - y_pred), axis=[1, 2, 3, 4])
    false_pos = tf.reduce_sum((1 - y_true) * y_pred, axis=[1, 2, 3, 4])
    
    tversky = (intersection + smooth) / (intersection + alpha * false_neg + beta * false_pos + smooth)
    
    return tf.reduce_mean(1 - tversky)



'''
使用 TensorFlow 实现 Sobel 边缘检测
'''
def sobel_filter_3d():
    # 创建 Sobel 滤波器，注意这里设置 input_depth 为 1
    filters = {
        'x': tf.constant([[[[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]],
                           [[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]],
                           [[[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]]]]], dtype=tf.float32),
        'y': tf.constant([[[[[[-1, -2, -1], [0, 0, 0], [1, 2, 1]]],
                           [[[-1, -2, -1], [0, 0, 0], [1, 2, 1]]],
                           [[[-1, -2, -1], [0, 0, 0], [1, 2, 1]]]]]], dtype=tf.float32),
        'z': tf.constant([[[[[[-1, -2, -1], [0, 0, 0], [1, 2, 1]]],
                           [[[-1, -2, -1], [0, 0, 0], [1, 2, 1]]],
                           [[[-1, -2, -1], [0, 0, 0], [1, 2, 1]]]]]], dtype=tf.float32)
    }
    
    # 过滤器的形状应该为 (depth, height, width, in_channels, out_channels)
    filters['x'] = tf.reshape(filters['x'], [3, 3, 3, 1, 1])
    filters['y'] = tf.reshape(filters['y'], [3, 3, 3, 1, 1])
    filters['z'] = tf.reshape(filters['z'], [3, 3, 3, 1, 1])

    return filters

def apply_sobel_filters(volume, filters):
    volume = tf.expand_dims(volume, axis=-1)  # Ensure volume has channel dimension
    
    conv_x = tf.nn.conv3d(volume, filters['x'], strides=[1, 1, 1, 1, 1], padding='SAME')
    conv_y = tf.nn.conv3d(volume, filters['y'], strides=[1, 1, 1, 1, 1], padding='SAME')
    conv_z = tf.nn.conv3d(volume, filters['z'], strides=[1, 1, 1, 1, 1], padding='SAME')

    edges = tf.sqrt(tf.square(conv_x) + tf.square(conv_y) + tf.square(conv_z))
    return edges

def compute_sobel_edges_3d(volume):
    filters = sobel_filter_3d()
    edges = apply_sobel_filters(volume, filters)
    return edges



'''
Boundary Loss
'''
# 欧几里得边界损失函数
@tf.function
def euclidean_distance_loss(true_edges, pred_edges):
    distance = tf.sqrt(tf.reduce_sum(tf.square(true_edges - pred_edges), axis=[1, 2, 3, 4]))
    return tf.reduce_mean(distance)

# 二进制交集边界损失函数
@tf.function
def binary_intersection_loss(true_edges, pred_edges):
    intersection = tf.reduce_sum(tf.cast(true_edges > 0, tf.float32) * tf.cast(pred_edges > 0, tf.float32), axis=[1, 2, 3, 4])
    return tf.reduce_mean(1 - intersection)


# 组合损失函数
@tf.function
def combined_loss(y_true, y_pred):
    
    # 确保 y_true 和 y_pred 是浮点型
    y_true = tf.cast(y_true, tf.float32)
    y_pred = tf.cast(y_pred, tf.float32)
    
    # 使用掩码来只关注标签1和标签2
    true_mask = tf.logical_or(tf.equal(y_true, 1), tf.equal(y_true, 2))
    pred_mask = tf.logical_or(tf.equal(y_pred, 1), tf.equal(y_pred, 2))
    
    # 将掩码应用于标签网格和预测网格
    y_true_masked = tf.where(true_mask, y_true, tf.zeros_like(y_true))
    y_pred_masked = tf.where(pred_mask, y_pred, tf.zeros_like(y_pred))
    
    # 计算真实边界和预测边界
    true_edges = compute_sobel_edges_3d(y_true_masked)
    pred_edges = compute_sobel_edges_3d(y_pred_masked)
    
    # 计算边界损失
    boundary_loss_euclidean = euclidean_distance_loss(true_edges, pred_edges)
    boundary_loss_intersection = binary_intersection_loss(true_edges, pred_edges)
    
    # 定义损失函数
    focal_loss_func = focal_loss(gamma=2.0, alpha=0.25)  # Create focal loss function
    focal = focal_loss_func(y_true, y_pred)  # Compute focal loss
    tversky = tversky_loss(y_true, y_pred)
    
    # 为每个损失函数赋权
    weight_focal = 1.0
    weight_tversky = 1.0
    weight_euclidean = 0.01
    weight_intersection = 1.0
    
    # 求和
    total_loss = (weight_focal * focal + 
                  weight_tversky * tversky + 
                  weight_euclidean * boundary_loss_euclidean +
                  weight_intersection * boundary_loss_intersection)
    
    return total_loss


In [6]:
import tensorflow as tf

# 加载模型，如果已经有模型可以不运行这个cell
model_load_path = r'July30语义分割模型/UNET_前四个损失函数_无数据增强.h5'
model = tf.keras.models.load_model(model_load_path)


TypeError: Error when deserializing class 'Conv3DTranspose' using config={'name': 'conv3d_transpose', 'trainable': True, 'dtype': 'float32', 'filters': 256, 'kernel_size': [2, 2, 2], 'strides': [2, 2, 2], 'padding': 'same', 'data_format': 'channels_last', 'groups': 1, 'activation': 'linear', 'use_bias': True, 'kernel_initializer': {'module': 'keras.initializers', 'class_name': 'GlorotUniform', 'config': {'seed': None}, 'registered_name': None}, 'bias_initializer': {'module': 'keras.initializers', 'class_name': 'Zeros', 'config': {}, 'registered_name': None}, 'kernel_regularizer': None, 'bias_regularizer': None, 'activity_regularizer': None, 'kernel_constraint': None, 'bias_constraint': None, 'output_padding': None}.

Exception encountered: Unrecognized keyword arguments passed to Conv3DTranspose: {'groups': 1}

In [ ]:
# 开启交互旋转
%matplotlib notebook


In [ ]:
'''
读取点云并对预测每点的类别标签
'''

'''
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from collections import defaultdict
from itertools import combinations

def load_obj_file(file_path):
    vertices = []
    faces = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('v '):
                vertices.append(list(map(float, line.strip().split()[1:])))
            elif line.startswith('f '):
                faces.append([int(idx.split('/')[0]) - 1 for idx in line.strip().split()[1:]])
    return np.array(vertices), np.array(faces)

def center_vertices(vertices):
    centroid = np.mean(vertices, axis=0)
    return vertices - centroid

def create_voxel_grid(points, grid_size):
    min_coords = np.min(points, axis=0)
    max_coords = np.max(points, axis=0)
    voxel_dim = (max_coords - min_coords) / grid_size
    
    voxel_grid = np.zeros((grid_size, grid_size, grid_size), dtype=np.int32)
    for point in points:
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, 0, grid_size-1)
        voxel_grid[voxel[0], voxel[1], voxel[2]] += 1
    
    return voxel_grid, min_coords, voxel_dim

def get_labels_from_model(model, voxel_grid):
    voxel_grid = np.expand_dims(voxel_grid, axis=0)  # Add batch dimension
    voxel_grid = np.expand_dims(voxel_grid, axis=-1)  # Add channel dimension
    predictions = model.predict(voxel_grid)
    
    labels = np.argmax(predictions, axis=-1)  # Get the class with the highest probability
    labels = labels.reshape(voxel_grid.shape[1], voxel_grid.shape[2], voxel_grid.shape[3])
    
    return labels

def apply_labels_to_point_cloud(data, predicted_labels, min_coords, voxel_dim, grid_size):
    labels = np.zeros(len(data))
    for i, point in enumerate(data):
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, 0, grid_size-1)
        labels[i] = predicted_labels[voxel[0], voxel[1], voxel[2]]
    return labels
'''

In [ ]:
import numpy as np
import tensorflow as tf
import dcrf
from dcrf import create_pairwise_gaussian, create_pairwise_bilateral, unary_from_softmax

def load_obj_file(file_path):
    vertices = []
    faces = []
    with open(file_path, 'r') as file:
        for line in file:
            if line.startswith('v '):
                vertices.append(list(map(float, line.strip().split()[1:])))
            elif line.startswith('f '):
                faces.append([int(idx.split('/')[0]) - 1 for idx in line.strip().split()[1:]])
    return np.array(vertices), np.array(faces)

def center_vertices(vertices):
    centroid = np.mean(vertices, axis=0)
    return vertices - centroid

def create_voxel_grid(points, grid_size):
    min_coords = np.min(points, axis=0)
    max_coords = np.max(points, axis=0)
    voxel_dim = (max_coords - min_coords) / grid_size
    
    voxel_grid = np.zeros((grid_size, grid_size, grid_size), dtype=np.int32)
    for point in points:
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, 0, grid_size-1)
        voxel_grid[voxel[0], voxel[1], voxel[2]] += 1
    
    return voxel_grid, min_coords, voxel_dim

def get_labels_from_model(model, voxel_grid):
    voxel_grid = np.expand_dims(voxel_grid, axis=0)  # Add batch dimension
    voxel_grid = np.expand_dims(voxel_grid, axis=-1)  # Add channel dimension
    predictions = model.predict(voxel_grid)
    
    labels = np.argmax(predictions, axis=-1)  # Get the class with the highest probability
    labels = labels.reshape(voxel_grid.shape[1], voxel_grid.shape[2], voxel_grid.shape[3])
    
    return labels

def upscale_voxel_grid(voxel_grid, new_grid_size):
    old_size = voxel_grid.shape[0]
    scale_factor = new_grid_size / old_size
    
    # Create an empty grid with new dimensions
    new_voxel_grid = np.zeros((new_grid_size, new_grid_size, new_grid_size), dtype=np.int32)
    
    # Use interpolation to fill the new grid
    for i in range(new_grid_size):
        for j in range(new_grid_size):
            for k in range(new_grid_size):
                # Compute the corresponding coordinates in the old grid
                old_i = int(i / scale_factor)
                old_j = int(j / scale_factor)
                old_k = int(k / scale_factor)
                
                # Use nearest neighbor interpolation for simplicity
                new_voxel_grid[i, j, k] = voxel_grid[old_i, old_j, old_k]
    
    return new_voxel_grid

def apply_crf(voxel_grid, probs):
    grid_size = voxel_grid.shape[0]
    num_voxels = grid_size * grid_size * grid_size

    # 创建 DenseCRF 对象
    d = dcrf.DenseCRF(num_voxels, 2)

    # 创建 Unary 能量
    probs = np.stack([1 - probs, probs], axis=0)  # Create the correct shape for probabilities
    probs = np.ascontiguousarray(probs)
    probs = probs.reshape((2, num_voxels))  # Reshape probabilities to (2, num_voxels)

    U = unary_from_softmax(probs)
    d.setUnaryEnergy(U)

    # 增加高斯对势能（基于位置的光滑项）
    pairwise_gaussian = create_pairwise_gaussian(sdims=(3, 3, 3), shape=(grid_size, grid_size, grid_size))
    d.addPairwiseEnergy(pairwise_gaussian, compat=3)

    # 增加双边对势能（基于颜色和位置的光滑项）
    voxel_grid = voxel_grid.reshape((grid_size, grid_size, grid_size, 1))  # Reshape voxel grid to add a channel dimension
    pairwise_bilateral = create_pairwise_bilateral(sdims=(3, 3, 3), schan=(1,), img=voxel_grid, chdim=3)
    d.addPairwiseEnergy(pairwise_bilateral, compat=10)

    # 进行推理
    Q = d.inference(5)
    Q = np.array(Q)  # 将 Q 转换为 NumPy 数组

    # 获得分割结果
    result = np.argmax(Q, axis=0).reshape((grid_size, grid_size, grid_size))

    return result

def apply_labels_to_point_cloud(data, predicted_labels, min_coords, voxel_dim, grid_size):
    labels = np.zeros(len(data))
    for i, point in enumerate(data):
        voxel = ((point - min_coords) / voxel_dim).astype(int)
        voxel = np.clip(voxel, 0, grid_size-1)
        labels[i] = predicted_labels[voxel[0], voxel[1], voxel[2]]
    return labels


In [ ]:
'''
给点云画拓扑图
'''

import numpy as np
import networkx as nx
from sklearn.neighbors import NearestNeighbors

def build_topology_graph(vertices, k=10):
    # 计算k近邻图
    nbrs = NearestNeighbors(n_neighbors=k, algorithm='auto').fit(vertices)
    distances, indices = nbrs.kneighbors(vertices)
    
    G = nx.Graph()
    for i, neighbors in enumerate(indices):
        for j in neighbors:
            if i != j:
                G.add_edge(i, j)
                
    return G

def largest_connected_component(G, nodes):
    subgraph = G.subgraph(nodes)
    largest_cc = max(nx.connected_components(subgraph), key=len)
    return subgraph.subgraph(largest_cc)

def get_max_connected_subgraphs(G, labels, target_labels):
    subgraphs = {}
    for label in target_labels:
        nodes = [i for i, l in enumerate(labels) if l == label]
        if nodes:
            largest_cc = largest_connected_component(G, nodes)
            subgraphs[label] = largest_cc
    return subgraphs



In [ ]:
'''
找1-2的边界线
'''

def find_boundary_edges(G, subgraph1, subgraph2):
    boundary_edges = []
    for edge in G.edges():
        if (edge[0] in subgraph1 and edge[1] in subgraph2) or (edge[0] in subgraph2 and edge[1] in subgraph1):
            boundary_edges.append(edge)
    return boundary_edges

def plot_surface_with_boundary_lines(vertices, faces, labels, boundary_edges, view_angles=(30, 30), angles=(0, 0, 0)):
    import matplotlib.pyplot as plt
    from mpl_toolkits.mplot3d.art3d import Line3DCollection

    fig = plt.figure()
    ax = fig.add_subplot(111, projection='3d')
    
    vertices = np.array(vertices)
    faces = np.array(faces)
    
    # Apply rotation
    vertices = rotate_points(vertices, angles)
    
    x, y, z = vertices.T
    
    # 绘制不同标签区域的点云
    colors = ['lightgreen', 'cornflowerblue', 'honeydew']
    for i in range(3):
        part_faces = [face for face in faces if sum(labels[vertex] == i for vertex in face) > 1]
        if len(part_faces) > 0:
            ax.plot_trisurf(vertices[:, 0], vertices[:, 1], vertices[:, 2], triangles=part_faces, color=colors[i], alpha=0.6)
    
    # 绘制边界线
    for edge in boundary_edges:
        p1, p2 = vertices[list(edge)]
        ax.plot([p1[0], p2[0]], [p1[1], p2[1]], [p1[2], p2[2]], color='red', lw=1)
    
    # 设置标签和标题
    ax.set_xlabel('X axis')
    ax.set_ylabel('Y axis')
    ax.set_zlabel('Z axis')
    ax.set_title('3D Model with Boundary Lines')

    # 确保坐标轴刻度一致
    max_range = np.array([max(x)-min(x), max(y)-min(y), max(z)-min(z)]).max()
    mid_x = (max(x) + min(x)) * 0.5
    mid_y = (max(y) + min(y)) * 0.5
    mid_z = (max(z) + min(z)) * 0.5
    ax.set_xlim(mid_x - max_range/2, mid_x + max_range/2)
    ax.set_ylim(mid_y - max_range/2, mid_y + max_range/2)
    ax.set_zlim(mid_z - max_range/2, mid_z + max_range/2)

    # 设置视角
    elev, azim = view_angles
    ax.view_init(elev=elev, azim=azim)  # Adjust these values as needed

    # 确保坐标轴比例相等
    ax.set_box_aspect([1,1,1])  # Aspect ratio is 1:1:1

    # 启用交互式旋转
    plt.show()


In [ ]:
'''
设置视角
'''

def rotate_points(points, angles):
    x_angle, y_angle, z_angle = angles
    Rx = np.array([
        [1, 0, 0],
        [0, np.cos(x_angle), -np.sin(x_angle)],
        [0, np.sin(x_angle), np.cos(x_angle)]
    ])
    Ry = np.array([
        [np.cos(y_angle), 0, np.sin(y_angle)],
        [0, 1, 0],
        [-np.sin(y_angle), 0, np.cos(y_angle)]
    ])
    Rz = np.array([
        [np.cos(z_angle), -np.sin(z_angle), 0],
        [np.sin(z_angle), np.cos(z_angle), 0],
        [0, 0, 1]
    ])
    R = Rz @ Ry @ Rx
    return points @ R.T


In [ ]:
'''
出图
'''

# 加载点云数据
obj_file_path = r'D:/李娅宁/肩台外侧点-0715/已完成预处理的原始数据/15/15_1.obj'
vertices, faces = load_obj_file(obj_file_path)
centered_vertices = center_vertices(vertices)

# 将点云转换为体素网格
grid_size = 16  # Ensure grid size matches model requirements
voxel_grid, min_coords, voxel_dim = create_voxel_grid(np.array(centered_vertices), grid_size)

# 使用训练好的模型进行预测
predicted_labels = get_labels_from_model(model, voxel_grid)

# 等比扩大体素网格
new_grid_size = 128  # Example new grid size, adjust as needed
upscaled_voxel_grid = upscale_voxel_grid(voxel_grid, new_grid_size)

# 使用CRF后处理
probs = np.array(predicted_labels, dtype=np.float32) / np.max(predicted_labels)  # Normalize probabilities
predicted_labels_crf = apply_crf(upscaled_voxel_grid, probs)

# 获取原始点云的预测标签
predicted_point_labels = apply_labels_to_point_cloud(np.array(centered_vertices), predicted_labels_crf, min_coords, voxel_dim, new_grid_size)

# 构建拓扑图
G = build_topology_graph(centered_vertices)

# 找到最大连通子图
target_labels = [1, 2]
subgraphs = get_max_connected_subgraphs(G, predicted_point_labels, target_labels)

# 找到边界线
boundary_edges = find_boundary_edges(G, subgraphs[1], subgraphs[2])

# 绘制点云及边界线
plot_surface_with_boundary_lines(centered_vertices, faces, predicted_point_labels, boundary_edges, view_angles=(30, 30), angles=(np.radians(90), np.radians(-30), np.radians(30)))
